In [6]:
# import sys
# !{sys.executable} -m pip install selenium

In [25]:
import requests, re, typing
from bs4 import BeautifulSoup as soup
import contextlib
import pandas as pd

In [8]:
def _remove(d:list) -> list:
    return list(filter(None, [re.sub('\xa0', '', b) for b in d]))

In [9]:
@contextlib.contextmanager
def get_weather_data(url:str, by_url = True) -> typing.Generator[dict, None, None]:
    d = soup(requests.get(url).text if by_url else url, 'html.parser')
    _table = d.find('table', {'id':'wt-his'})
    _data = [[[i.text for i in c.find_all('th')], *[i.text for i in c.find_all('td')]] for c in _table.find_all('tr')]
    [h1], [h2], *data, _ = _data
    _h2 = _remove(h2)
    yield {tuple(_remove(h1)):[dict(zip(_h2, _remove([a, *i]))) for [[a], *i] in data]}

In [10]:
year = 2023
month = 3

with get_weather_data(f'https://www.timeanddate.com/weather/usa/dayton/historic?month={month}&year={year}') as weather:
    print(weather)

{('Conditions', 'Comfort'): [{'Time': '12:55 amWed, 1 Mar', 'Temp': '4°C', 'Weather': 'Clear.', 'Wind': '11 km/h', 'Humidity': '↑', 'Barometer': '79%', 'Visibility': '1012 mbar'}, {'Time': '1:55 am', 'Temp': '4°C', 'Weather': 'Clear.', 'Wind': '11 km/h', 'Humidity': '↑', 'Barometer': '80%', 'Visibility': '1011 mbar'}, {'Time': '2:55 am', 'Temp': '4°C', 'Weather': 'Passing clouds.', 'Wind': '13 km/h', 'Humidity': '↑', 'Barometer': '82%', 'Visibility': '1010 mbar'}, {'Time': '3:55 am', 'Temp': '4°C', 'Weather': 'Clear.', 'Wind': '11 km/h', 'Humidity': '↑', 'Barometer': '82%', 'Visibility': '1008 mbar'}, {'Time': '4:55 am', 'Temp': '4°C', 'Weather': 'Clear.', 'Wind': '9 km/h', 'Humidity': '↑', 'Barometer': '82%', 'Visibility': '1007 mbar'}, {'Time': '5:55 am', 'Temp': '3°C', 'Weather': 'Clear.', 'Wind': '7 km/h', 'Humidity': '↑', 'Barometer': '85%', 'Visibility': '1007 mbar'}, {'Time': '6:55 am', 'Temp': '2°C', 'Weather': 'Clear.', 'Wind': 'No wind', 'Humidity': '↑', 'Barometer': '90%', '

In [71]:
def get_daily_data(weather):
    df_day = pd.DataFrame()
    for output in weather.values():
        for row in output:
            df_hour = pd.DataFrame(row, index=[0])
            df_day = df_day.append(df_hour)
    return df_day

df_day = get_daily_data(weather)
df_day

,Time,Temp,Weather,Wind,Humidity,Barometer,Visibility
0,"12:55 amWed, 1 Mar",4°C,Clear.,11 km/h,↑,79%,1012 mbar
0,1:55 am,4°C,Clear.,11 km/h,↑,80%,1011 mbar
0,2:55 am,4°C,Passing clouds.,13 km/h,↑,82%,1010 mbar
0,3:55 am,4°C,Clear.,11 km/h,↑,82%,1008 mbar
0,4:55 am,4°C,Clear.,9 km/h,↑,82%,1007 mbar
0,5:55 am,3°C,Clear.,7 km/h,↑,85%,1007 mbar
0,6:55 am,2°C,Clear.,No wind,↑,90%,1008 mbar
0,7:55 am,3°C,Sunny.,No wind,↑,87%,1008 mbar
0,8:55 am,8°C,Sunny.,No wind,↑,75%,1008 mbar
0,9:55 am,12°C,Passing clouds.,17 km/h,↑,62%,1009 mbar
